In [1]:
import os
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
import shutil
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential
from keras.models import load_model
import splitfolders
import sklearn.metrics
import numpy as np
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

In [2]:
def create_df():
    w=[]
    classes=[]
    uk=[]

    path=r'D:\SEM 8\Miniproject\CODE\Data'
    for files in os.listdir(path):
        for p in os.listdir(path+"\\"+files):
            w.append(path+"\\"+files+"\\"+p)
            classes.append(files)

    df=pd.DataFrame({'Image':w,'Class':classes})
    for i in range(len(df)):
        if df['Class'][i]=='Unknown':
            uk.append('Unknown')
        else:
            uk.append('Known')
    df['U/K']=uk
    return df

df=create_df()
df

,Image,Class,U/K
0,D:\SEM 8\Miniproject\CODE\Data\Abdullah_Gul\11...,Abdullah_Gul,Known
1,D:\SEM 8\Miniproject\CODE\Data\Abdullah_Gul\12...,Abdullah_Gul,Known
2,D:\SEM 8\Miniproject\CODE\Data\Abdullah_Gul\12...,Abdullah_Gul,Known
3,D:\SEM 8\Miniproject\CODE\Data\Abdullah_Gul\14...,Abdullah_Gul,Known
4,D:\SEM 8\Miniproject\CODE\Data\Abdullah_Gul\21...,Abdullah_Gul,Known
...,...,...,...
10459,D:\SEM 8\Miniproject\CODE\Data\Yoriko_Kawaguch...,Yoriko_Kawaguchi,Known
10460,D:\SEM 8\Miniproject\CODE\Data\Yoriko_Kawaguch...,Yoriko_Kawaguchi,Known
10461,D:\SEM 8\Miniproject\CODE\Data\Yoriko_Kawaguch...,Yoriko_Kawaguchi,Known
10462,D:\SEM 8\Miniproject\CODE\Data\Yoriko_Kawaguch...,Yoriko_Kawaguchi,Known


In [3]:
path=r"D:\SEM 8\Miniproject\CODE\splits"
if os.path.exists(path):
    shutil.rmtree(path)

In [4]:
splitfolders.ratio(r"D:\SEM 8\Miniproject\CODE\Data", output="D:\SEM 8\Miniproject\CODE\splits", seed=1337, ratio=(.8,.1,.1)) 

Copying files: 10464 files [00:11, 947.91 files/s] 


In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [3]:
train = train_datagen.flow_from_directory(r"D:\SEM 8\Miniproject\CODE\splits\train", target_size = (224,224), batch_size = 64, color_mode = 'rgb', class_mode = 'categorical')
test = test_datagen.flow_from_directory(r"D:\SEM 8\Miniproject\CODE\splits\test", target_size = (224,224), batch_size = 64, color_mode = 'rgb', class_mode = 'categorical')

Found 8305 images belonging to 159 classes.
Found 1185 images belonging to 159 classes.


In [4]:
def train_m(opt,train_model,nodes,dropout,num_class,actication,epochs,lr):
    model=Sequential([train_model,Flatten(),Dense(nodes, activation = 'relu'),Dropout(dropout),
                      Dense(num_class, activation = activation)])
    model.summary()
    if opt=="adam":
        optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train, validation_data = test, epochs = epochs)
    return history,model

In [5]:
nodes = 256
dropout = 0.4
activation = 'softmax'
epochs = 6
lr = 0.0001
num_class=159
opt="adam"

### RESNET152V2

In [1]:
train_model = keras.applications.ResNet152V2(include_top = False, input_shape = (224,224, 3), weights = 'imagenet', pooling = 'max')

NameError: name 'keras' is not defined

In [16]:
history,model=train_m(opt,train_model,nodes,dropout,num_class,activation,epochs,lr)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 2048)              58331648  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 159)               40863     
                                                                 
Total params: 58,897,055
Trainable params: 58,753,311
Non-trainable params: 143,744
_________________________________________________________________
Epoch 1/6
130/130 [=================

In [17]:
model.save("Resnet152v2-1.h5")

In [8]:
resnet152=load_model("D:\SEM 8\Miniproject\CODE\Resnet152v2.h5")
p1=resnet152.predict(test)

29/29 [==============================] - 317s 11s/step


In [12]:
p1[0]

array([2.96894349e-08, 8.78035422e-08, 3.56072661e-07, 1.48400385e-08,
       5.05208675e-08, 1.92199749e-08, 8.73402044e-08, 9.61913429e-07,
       1.95205621e-06, 4.87019975e-07, 1.12610394e-06, 2.48001982e-07,
       1.25257395e-06, 1.34390666e-06, 1.04823030e-07, 2.01152552e-08,
       6.01054282e-08, 2.08330135e-07, 2.86469259e-07, 1.42866963e-08,
       6.48079634e-09, 5.42122214e-09, 1.35153030e-07, 2.40430520e-08,
       3.72773421e-08, 3.19321384e-06, 1.05726139e-08, 5.79838888e-09,
       3.64639234e-08, 5.39675327e-08, 2.98208782e-08, 9.66901559e-10,
       7.61789295e-07, 5.33027162e-07, 1.58260391e-07, 1.24983546e-06,
       3.10602918e-05, 8.09113665e-09, 1.76105495e-08, 4.41466227e-07,
       3.02754273e-07, 1.64119740e-06, 6.95773821e-08, 1.45343591e-07,
       1.54145212e-08, 5.34732543e-08, 1.83528108e-08, 4.52317934e-08,
       3.05257595e-06, 3.43770324e-07, 1.68320771e-08, 1.01002087e-07,
       3.35512684e-09, 5.39000268e-07, 2.53162654e-07, 5.39107702e-07,
      

In [21]:
print("Accuracy of ResNet152V2 model: ",model.evaluate(test)[1])

26/26 [==============================] - 275s 11s/step - loss: 2.6839 - accuracy: 0.6212
Accuracy of ResNet152V2 model:  0.62123042345047


In [10]:
y_result = []
y_actual = []
for p in p1:
    y_result.append(np.argmax(p))
for i in range(len(test)):
    for p in test[i][1]:
        y_actual.append(np.argmax(p))

In [11]:
cm=sklearn.metrics.confusion_matrix(y_actual, y_result)
df1=pd.DataFrame(cm)
df1

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,158
0,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0,0,0,0,0,0,0,0,0,0,...,0,0,7,0,0,0,0,0,0,0
155,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
156,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
157,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0


### VGG-19

In [6]:
train_model = keras.applications.VGG19(include_top = False, input_shape = (224,224, 3), weights = 'imagenet', pooling = 'max')

In [ ]:
history,model=train_m(opt,train_model,nodes,dropout,num_class,activation,epochs,lr)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 512)               20024384  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 159)               40863     
                                                                 
Total params: 20,196,575
Trainable params: 20,196,575
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
130/130 [=========================

In [10]:
model.save("vgg19-1.h5")
vgg19=load_model("vgg19-1.h5")
p2=vgg19.predict(test)

26/26 [==============================] - 229s 9s/step


In [12]:
print("Accuracy of VGG-19 model: ",model.evaluate(test)[1])

26/26 [==============================] - 280s 11s/step - loss: 2.8940 - accuracy: 0.5489
Accuracy of VGG-19 model:  0.5488540530204773


In [ ]:
y_result = []
y_actual = []
for p in p2:
    y_result.append(np.argmax(p))
for i in range(len(test)):
    for p in test[i][1]:
        y_actual.append(np.argmax(p))

In [ ]:
cm=sklearn.metrics.confusion_matrix(y_actual, y_result)
df2=pd.DataFrame(cm)
df2

### DENSENET169

In [6]:
train_model = keras.applications.DenseNet169(include_top = False, input_shape = (224,224, 3), weights = 'imagenet', pooling = 'max')

In [7]:
history,model=train_m(opt,train_model,nodes,dropout,num_class,activation,epochs,lr)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet169 (Functional)    (None, 1664)              12642880  
                                                                 
 flatten (Flatten)           (None, 1664)              0         
                                                                 
 dense (Dense)               (None, 256)               426240    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 159)               40863     
                                                                 
Total params: 13,109,983
Trainable params: 12,951,583
Non-trainable params: 158,400
_________________________________________________________________
Epoch 1/6
130/130 [===================

In [8]:
model.save("densenet-1.h5")

In [10]:
densenet=load_model("densenet.h5")
p3=densenet.predict(test)

26/26 [==============================] - 95s 4s/step


In [11]:
print("Accuracy of DenseNet169 model: ",model.evaluate(test)[1])

26/26 [==============================] - 111s 4s/step - loss: 2.0000 - accuracy: 0.6538
Accuracy of DenseNet169 model:  0.6537997722625732


In [13]:
y_result = []
y_actual = []
for p in p3:
    y_result.append(np.argmax(p))
for i in range(len(test)):
    for p in test[i][1]:
        y_actual.append(np.argmax(p))

In [14]:
cm=sklearn.metrics.confusion_matrix(y_actual, y_result)
df3=pd.DataFrame(cm)
df3

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,158
0,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0
155,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
156,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
157,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
